In [0]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

In [0]:
### Importing tables
SAMPLE_MATERIALS = spark.read.table('default.sample_material_list')\
                        .select('MATNR','AREA','TYPE','LGORT')
SAP_EBAN = spark.read.table('default.sap_eban')\
                .withColumn('BNFPO',col('BNFPO').cast(IntegerType()).cast(StringType()))

SAP_EKPO = spark.read.table('default.sap_ekpo')\
                .withColumn('BNFPO',col('BNFPO').cast(IntegerType()).cast(StringType()))\
                .withColumn('EBELP',col('EBELP').cast(IntegerType()).cast(StringType()))

SAP_EKKO = spark.read.table('default.sap_ekko').distinct()\
                .withColumn('AEDAT',expr("TO_DATE(AEDAT,'yyyyMMdd')"))\
                .withColumnRenamed('AEDAT','POCREATE')

SAP_EKES = spark.read.table('default.sap_ekes')\
                .filter(col('EBTYP') == 'AB')\
                .withColumn('EBELP',col('EBELP').cast(IntegerType()).cast(StringType()))\
                .withColumnRenamed('EINDT','POA')\
                .withColumn('POA',expr("TO_DATE(POA,'yyyyMMdd')"))\
                .withColumn('ERDAT',expr("TO_DATE(ERDAT,'yyyyMMdd')"))\
                
SAP_EKBE = spark.read.table('default.sap_ekbe')\
                .filter(col('BEWTP') == 'E')\
                .withColumn('EBELP',col('EBELP').cast(IntegerType()).cast(StringType()))\
                .withColumnRenamed('MENGE','RECEIPT_QTY')\
                .withColumn('BUDAT',expr("TO_DATE(BUDAT,'yyyyMMdd')"))\
                .withColumnRenamed('BUDAT','RECEIPT_DATE')

SAP_EKET = spark.read.table('default.sap_eket')\
                .withColumn('EBELP',col('EBELP').cast(IntegerType()).cast(StringType()))\
                .withColumn('EINDT',expr("TO_DATE(EINDT,'yyyyMMdd')"))\
                .withColumnRenamed('EINDT','PO_RAS')
                
SAP_MARA = spark.read.table('default.sap_mara')
SAP_MARD = spark.read.table('default.sap_mard')
SAP_MBEW = spark.read.table('default.sap_mbew')
SAP_LFA1 = spark.read.table('default.sap_lfa1')\
                .filter(col('KTOKK').isin(['ZVEN','ZLAT']))

In [0]:
### OPEN PO Table
OPEN_PO = SAP_EKPO.join(SAP_EKKO,['EBELN'],'inner')\
                  .join(SAP_EKBE,['EBELN','EBELP'],'left')\
                  .join(SAP_EKES,['EBELN','EBELP'],'left')\
                  .join(SAP_EKET,['EBELN','EBELP'],'left')\
                  .withColumn('POLINE',col('EBELP').cast(IntegerType()).cast(StringType()))\
                  .withColumn('MATERIAL',when(substring('MATNR',1,1) == 'R',col('MATNR'))\
                                        .otherwise(substring('MATNR',-7,7)))\
                  .withColumn('AREA_finding',concat(SAP_EKPO.WERKS,SAP_EKPO.LGORT))\
                  .withColumn('AREA',
                              when(col('AREA_finding').isin(['90021000','90031000','90041000','90051000','90061000',
                                                             '90071000','90101000','90111000']),'NPI')\
                              .when(col('AREA_finding').isin(['9000CPEQ','9001CPEQ','9002CPEQ','9003CPEQ',
                                                              '9004CPEQ','9005CPEQ','9006CPEQ','9007CPEQ']),'MGD SER')\
                              .when(col('AREA_finding').isin(['55001000','55011000','55021000',
                                                              '55031000','55041000']),'CONSUMER')\
                              .when(col('AREA_finding').isin(['19101000','19201000','19211000','19231000',
                                                              '16001000','17001000','18001000']),'HARDWIRE')\
                              .when(col('AREA_finding') == '19011000','PICS')\
                              .when(col('AREA_finding').isin(['90521000','9052PROJ']),'EMEA')\
                              .when(col('AREA_finding') == '9052CPEQ','EMEA CPEQ')\
                              .when(col('AREA_finding') == '9052SPAR','EMEA SPAR')\
                              .when(col('AREA_finding').isin(['9002SPAR','9003SPAR','9004SPAR','9005SPAR','9006SPAR',
                                                          '9007SPAR','9000SPAR','9001SPAR','9010SPAR','9011SPAR']),'NPI SPARE')\
                              .when(col('AREA_finding') == '8000SPAR','DEPOT SPARE')\
                              .when(col('AREA_finding') == '1906CCTS','NA CTS'))\
                  .withColumn('NEWMATERIAL',substring('MATNR',-7,7))\
                  .withColumn('POA',when(col('POA').isNull(),'2099-01-01')\
                                   .otherwise(col('POA')))\
                  .withColumn('POA',col('POA').cast(DateType()))\
                  .withColumn('RECEIPT_QTY',when(col('BWART') == '102',-col('RECEIPT_QTY'))\
                                     .otherwise(col('RECEIPT_QTY')))\
                  .withColumn('RECEIPT_QTY',when(col('RECEIPT_QTY').isNull(),0)\
                                     .otherwise(col('RECEIPT_QTY')))\
                  .filter((col('WERKS').isin(['9000','5503','9001','9011','1800','1700','1250','1600','9010','1921','5504',
                                              '5502','9003','9005','1900','7863','1920','5500','9002','1923','1901','1910',
                                              '5501','9004','9006','1922','9007']))
                          &(col('LGORT').isin(['1000','CPEQ'])
                          &(col('LOEKZ').isNull()))
                          &(col('POCREATE') > '2018-08-01'))\
                  .filter((col('BSART').isin(['CE','NB','AC','ZINV']))
                          &(col('ELIKZ').isNull()))\
                  .groupBy(col('EBELN').alias('PO'),'POLINE',col('BSART').alias('TYPE'),'MATERIAL',
                           col('WERKS').alias('PLANT'),col('LGORT').alias('STORAGE'),'AREA','NEWMATERIAL',
                           'POCREATE',col('MENGE').alias('PO_QTY'))\
                  .agg(max('POA').alias('POA'),
                      sum('RECEIPT_QTY').alias('RECEIPT_QTY'),
                      max('PO_RAS').alias('PO_RAS'),
                      sum('NETWR').alias('LINE_VALUE'))\
                  .withColumn('OPEN_QTY',col('PO_QTY') - col('RECEIPT_QTY'))\
                  .withColumn('OPEN_QTY',when(col('OPEN_QTY').isNull(),col('PO_QTY'))\
                                        .otherwise(col('OPEN_QTY')))\
                  .filter(col('OPEN_QTY') > 0)\
                  .withColumn('OPEN_VALUE',round(col('OPEN_QTY')*(col('LINE_VALUE')/col('PO_QTY')),0))\
                  .select('PO','POLINE','TYPE','MATERIAL','PLANT','STORAGE','AREA','NEWMATERIAL','POCREATE',
                          'POA','PO_QTY','RECEIPT_QTY','OPEN_QTY','PO_RAS','LINE_VALUE','OPEN_VALUE')

In [0]:
display(OPEN_PO)

PO,POLINE,TYPE,MATERIAL,PLANT,STORAGE,AREA,NEWMATERIAL,POCREATE,POA,PO_QTY,RECEIPT_QTY,OPEN_QTY,PO_RAS,LINE_VALUE,OPEN_VALUE
4504191102,1,NB,1223406,9011,1000,NPI,1223406,2023-08-10,2099-01-01,65,0.0,65.0,2024-01-05,1050.4,1050.0
4503921188,1,NB,1415603,9003,1000,NPI,1415603,2022-04-28,2099-01-01,181,0.0,181.0,2024-07-08,125433.0,125433.0
4504158752,1,NB,1223406,9011,1000,NPI,1223406,2023-06-14,2099-01-01,54,0.0,54.0,2023-11-03,872.64,873.0
4503920945,1,NB,1426657,9011,1000,NPI,1426657,2022-04-28,2099-01-01,102,0.0,102.0,2024-05-10,63036.0,63036.0
4504190456,1,NB,1415319,9011,1000,NPI,1415319,2023-08-09,2099-01-01,117,0.0,117.0,2023-11-10,5733.0,5733.0
4503920940,1,NB,1426657,9003,1000,NPI,1426657,2022-04-28,2099-01-01,102,0.0,102.0,2024-04-19,63036.0,63036.0
4503921084,1,NB,1426657,9011,1000,NPI,1426657,2022-04-28,2099-01-01,138,0.0,138.0,2024-03-15,85284.0,85284.0
4503871431,1,NB,1415244,9011,1000,NPI,1415244,2022-02-14,2022-08-05,61,0.0,61.0,2023-11-03,20008.0,20008.0
4504206662,3,NB,1269877,9003,1000,NPI,1269877,2023-09-11,2023-11-20,5,0.0,5.0,2023-11-20,400.0,400.0
4503921014,1,NB,1426745,9003,1000,NPI,1426745,2022-04-28,2099-01-01,207,0.0,207.0,2024-02-16,10350.0,10350.0


In [0]:
### Inventory_Table
INVENTORY = SAP_MARA.join(SAP_MARD,['MATNR'],'inner')\
                    .join(SAP_MBEW,(SAP_MBEW.MATNR == SAP_MARA.MATNR)
                                  &(SAP_MBEW.BWKEY == SAP_MARD.WERKS),'inner')\
                    .withColumn('MATERIAL',when(substring(SAP_MARA.MATNR,1,1) == 'R',SAP_MARA.MATNR)\
                                          .otherwise(substring(SAP_MARA.MATNR,-7,7)))\
                    .withColumn('NEW_MATC',substring(SAP_MARA.MATNR,-7,7))\
                    .withColumn('QUANTITY',round(col('LABST')))\
                    .withColumn('INVENTORY_TYPE',when(substring(SAP_MARA.MATNR,1,1) == '0','NEW')\
                                                .when(substring(SAP_MARA.MATNR,1,1) == 'R','USED')\
                                                .when(substring(SAP_MARA.MATNR,1,1) == '7','SERVICE')\
                                                .when(substring(SAP_MARA.MATNR,1,1) == 'C','CUSTOMER')\
                                                .otherwise('OTHER'))\
                    .withColumn('MOVING_AVG_PER',when(col('PEINH') == 0 , 1)\
                                                .otherwise(col('PEINH')))\
                    .withColumn('TOTAL_VALUATION',(col('VERPR')/col('MOVING_AVG_PER'))*col('QUANTITY'))\
                    .distinct().filter(col('LABST') > 0)\
                    .select('MATERIAL','NEW_MATC','QUANTITY',col('WERKS').alias('PLANT'),col('LGORT').alias('STORAGE_BIN'),
                            'INVENTORY_TYPE','TOTAL_VALUATION')

In [0]:
display(INVENTORY)

MATERIAL,NEW_MATC,QUANTITY,PLANT,STORAGE_BIN,INVENTORY_TYPE,TOTAL_VALUATION
R1415603,1415603,1.0,9848,SPAR,USED,0.01
1166284,1166284,2.0,983Z,2000,NEW,51.0
R1415603,1415603,1.0,9861,SPAR,USED,0.12
1166284,1166284,89.0,9045,1000,NEW,9078000.0
R1415319,1415319,1.0,9001,AWRS,USED,0.01
R1269878,1269878,1.0,9931,SPAR,USED,0.01
R1166284,1166284,1.0,9467,SPAR,USED,1.2
R1223406,1223406,7.0,9003,SPAR,USED,0.0
R1415603,1415603,1.0,9844,SPAR,USED,6.29
R1166284,1166284,2.0,9354,SPAR,USED,0.2


In [0]:
###LEADTIME_table
CURRENT_DATE = '2023-10-11' ## Date on which data was collected 
POA = SAP_EKES.filter((col('POA') > '2015-01-01')
                     &(col('ERDAT') > '2015-01-01'))\
              .groupBy('EBELN','EBELP')\
              .agg(max('POA').alias('LAST_DATE'))\
              .withColumn('LAST_POA_DATE',expr("TO_DATE(LAST_DATE,'yyyyMMdd')"))\

LEADTIME = SAP_EKBE.join(SAP_EKPO,['EBELN','EBELP'],'inner')\
                   .join(SAP_EKKO,['EBELN'],'inner')\
                   .join(SAP_LFA1,SAP_EKKO.LIFNR == SAP_LFA1.LIFNR,'inner')\
                   .join(SAP_EBAN,['BANFN','BNFPO'],'left')\
                   .join(SAP_EKET.filter(col('PO_RAS') > '2015-01-01'),['EBELN','EBELP'],'left')\
                   .join(POA,['EBELN','EBELP'],'left')\
                   .withColumn('MATERIAL',when(substring(SAP_EKPO.MATNR,1,1) == 'R',SAP_EKPO.MATNR)\
                                         .otherwise(substring(SAP_EKPO.MATNR,-7,7)))\
                   .withColumnRenamed('POCREATE','PO_DATE')\
                   .withColumn('DAYS',datediff('RECEIPT_DATE','PO_DATE'))\
                   .groupBy('RECEIPT_DATE',SAP_EKBE.EBELN.alias('PO'),SAP_EKBE.EBELP.alias('PO_LINE'),
                            SAP_EKPO.WERKS.alias('PLANT'),'MATERIAL','PO_DATE',SAP_EKPO.BANFN.alias('REQID'),
                            SAP_EKPO.BNFPO.alias('REQ_LINE'),SAP_EBAN.BADAT.alias('REQ_DATE'),
                            SAP_EBAN.BSART.alias('REQ_TYPE'),'DAYS','LAST_POA_DATE')\
                   .agg(sum('RECEIPT_QTY').alias('RECEIPT_QTY'),
                        max('PO_RAS').alias('PO_RAS'))\
                   .withColumn('CURRENT_DATE',lit(CURRENT_DATE).cast(DateType()))\
                   .withColumn('DAYS_BETWEEN',datediff('CURRENT_DATE','RECEIPT_DATE'))\
                   .filter(col('DAYS_BETWEEN') < 730).drop('CURRENT_DATE','DAYS_BETWEEN')\
                   .select('RECEIPT_DATE','PO','PO_LINE','RECEIPT_QTY','PLANT','MATERIAL','PO_DATE','PO_RAS','REQID',
                           'REQ_LINE','REQ_DATE','REQ_TYPE','DAYS','LAST_POA_DATE')

In [0]:
display(LEADTIME)

RECEIPT_DATE,PO,PO_LINE,RECEIPT_QTY,PLANT,MATERIAL,PO_DATE,PO_RAS,REQID,REQ_LINE,REQ_DATE,REQ_TYPE,DAYS,LAST_POA_DATE
2022-08-28,4503993998,1,26.0,5204,1166284,2022-08-23,2022-08-31,7000542802,3,20220823,ZC,5,2022-08-31
2023-02-14,4503892764,1,16.0,9011,1426657,2022-03-16,2022-09-02,1003961604,1,20220310,ZINV,335,2022-09-02
2021-10-29,4503799784,1,1.0,5242,1166284,2021-10-08,2021-10-22,7000512202,9,20211008,ZC,21,2021-10-22
2022-10-21,4503994989,6,61.0,1901,1269877,2022-08-24,2022-11-03,1004035978,6,20220816,ZINV,58,null
2022-09-02,4503994666,1,2.0,7660,1166284,2022-08-24,2022-09-09,7000542823,5,20220824,ZC,9,null
2022-09-21,4503995249,1,2.0,7887,1166284,2022-08-25,2022-09-02,7000542907,2,20220824,ZC,27,2022-09-02
2022-02-12,4503842507,2,4.0,9639,1166284,2021-12-29,2022-01-31,1003926176,2,20211229,NB,45,2022-01-31
2023-08-16,4504187662,2,1.0,5270,1269877,2023-08-04,2023-08-07,7000575524,2,20230803,ZC,12,2023-08-07
2023-05-28,4504132544,2,2.0,9662,1166284,2023-04-28,2023-05-01,1004147786,2,20230428,NB,30,2023-05-01
2023-01-31,4504080809,1,2.0,7877,1166284,2023-01-26,2023-02-03,7000555885,3,20230126,ZC,5,2023-02-03


In [0]:
### BO_Details
BACKORDER = SAP_EBAN.withColumn('BADAT',expr("TO_DATE(BADAT,'yyyyMMdd')"))\
                    .filter((col('ZZ_PROCESSED').isin(['ONBO', 'STOP','STAV']))
                           &(col('BSART').isin(['NB', 'ZC']))
                           &(col('LOEKZ') != 'X')
                           &(~col('EKGRP').isin(['LEM','LAP']))
                           &((((col('EBELN') == ''))|(col('EBELN').isNull()))
                            |((col('EBELP').isNotNull()) &(col('MENGE') > col('BSMNG'))))
                           &(col('EBAKZ') != 'X')
                           &(col('BADAT') >= '2021-01-01')
                           &(~col('TXZ01').like('%SANITIZER%'))&(~col('TXZ01').like('%RESPIRATOR%'))
                           &(~col('TXZ01').like('%GLOVE%'))&(~col('TXZ01').like('%SOAP%'))
                           &(~col('TXZ01').like('%MASK%'))&(~col('TXZ01').like('%ALCOHOL%'))
                           &(~col('TXZ01').like('%DISINFECT%'))&(~col('TXZ01').like('%FACE COVER%'))
                           &(~col('TXZ01').like('%FACECVR%')))\
                    .withColumn('MATNR',substring(col('MATNR'),-7,7))\
                    .withColumn('PLANT_CONVERT',expr("CAST(WERKS AS INT)"))\
                    .withColumn('PLANT_CONVERT',when(col('PLANT_CONVERT').isNull(),col('WERKS'))
                                               .otherwise(col('PLANT_CONVERT')))\
                    .withColumn('TYPE',when(col('PLANT_CONVERT')>= 9000,'RED')
                                      .otherwise('GREEN'))\
                    .join(SAMPLE_MATERIALS.filter(col('LGORT') == '1000'),['MATNR','TYPE'],'left')\
                    .select(col('BANFN').alias('PR_NUMBER'),col('BNFPO').cast(IntegerType()).cast(StringType()).alias('PR_LINE'),
                            col('BADAT').alias('PR_DATE'),(col('MENGE')-col('BSMNG')).alias('QTY'),
                            col('MATNR').alias('MATCODE'),col('WERKS').alias('PLANT'),'AREA','ZZ_PROCESSED')

In [0]:
display(BACKORDER)

PR_NUMBER,PR_LINE,PR_DATE,QTY,MATCODE,PLANT,AREA,ZZ_PROCESSED
1004214110,2,2023-10-09,1.0,1269878,9639,NPI,ONBO
